In [2]:
import dac
from dac.model.dac import EncoderBlock, DecoderBlock, ResidualUnit
from dac.nn.layers import Snake1d
from audiotools import AudioSignal
import torch
from torch import nn
import copy

CUDA extension for structured kernels (Cauchy and Vandermonde multiplication) not found. Install by going to extensions/kernels/ and running `python setup.py install`, for improved speed and memory efficiency. Note that the kernel changed for state-spaces 4.0 and must be recompiled.
Falling back on slow Cauchy and Vandermonde kernel. Install at least one of pykeops or the CUDA extension for better speed and memory efficiency.


In [3]:
model_path = dac.utils.download(model_type="44khz")
model = dac.DAC.load(model_path)
print("done loading")

[codebook size] 1024
[# codebooks] 9
[ignore left crop] False
[encoder causal] False
[encoder frame indep] False
[decoder causal] False
[decoder frame indep] False
[# trainable params] 76651890
[# trainable params (enc)] 22307968
[# trainable params (dec)] 54104162
done loading


In [4]:
model.eval()

DAC(
  (encoder): Encoder(
    (block): Sequential(
      (0): Conv1d(1, 64, kernel_size=(7,), stride=(1,), padding=(3,))
      (1): EncoderBlock(
        (block): Sequential(
          (0): ResidualUnit(
            (block): Sequential(
              (0): Snake1d()
              (1): Conv1d(64, 64, kernel_size=(7,), stride=(1,), padding=(3,))
              (2): Snake1d()
              (3): Conv1d(64, 64, kernel_size=(1,), stride=(1,))
            )
          )
          (1): ResidualUnit(
            (block): Sequential(
              (0): Snake1d()
              (1): Conv1d(64, 64, kernel_size=(7,), stride=(1,), padding=(9,), dilation=(3,))
              (2): Snake1d()
              (3): Conv1d(64, 64, kernel_size=(1,), stride=(1,))
            )
          )
          (2): ResidualUnit(
            (block): Sequential(
              (0): Snake1d()
              (1): Conv1d(64, 64, kernel_size=(7,), stride=(1,), padding=(27,), dilation=(9,))
              (2): Snake1d()
              

In [ ]:
weight_bitwidth = 8
class QuantizedConv1d(nn.Module):
    def __init__(self, weight, bias,
                 input_zero_point, output_zero_point,
                 input_scale, weight_scale, output_scale,
                 stride, padding, dilation, groups,
                 feature_bitwidth=8, weight_bitwidth=8):
        super().__init__()
        # current version Pytorch does not support IntTensor as nn.Parameter
        self.register_buffer('weight', weight)
        self.register_buffer('bias', bias)

        self.input_zero_point = input_zero_point
        self.output_zero_point = output_zero_point

        self.input_scale = input_scale
        self.register_buffer('weight_scale', weight_scale)
        self.output_scale = output_scale

        self.stride = stride
        self.padding = (padding[1], padding[1], padding[0], padding[0])
        self.dilation = dilation
        self.groups = groups

        self.feature_bitwidth = feature_bitwidth
        self.weight_bitwidth = weight_bitwidth


    def forward(self, x):
        return quantized_conv2d(
            x, self.weight, self.bias,
            self.feature_bitwidth, self.weight_bitwidth,
            self.input_zero_point, self.output_zero_point,
            self.input_scale, self.weight_scale, self.output_scale,
            self.stride, self.padding, self.dilation, self.groups
            )

class QuantizedMaxPool2d(nn.MaxPool2d):
    def forward(self, x):
        # current version PyTorch does not support integer-based MaxPool
        return super().forward(x.float()).to(torch.int8)

class QuantizedAvgPool2d(nn.AvgPool2d):
    def forward(self, x):
        # current version PyTorch does not support integer-based AvgPool
        return super().forward(x.float()).to(torch.int8)

In [ ]:
quantized_model = copy.deepcopy(model_fused)


In [8]:
audio_file_path = "f1_script1_iphone_balcony1.wav"
signal = AudioSignal(audio_file_path)

signal = signal.cpu()
print(signal.shape)
# x = model.compress(signal)

# # Save and load to and from disk
# x.save("compressed.dac")
# x = dac.DACFile.load("compressed.dac")

# # Decompress it back to an AudioSignal
# y = model.decompress(x)



torch.Size([1, 1, 7539712])


In [5]:
# model.qconfig = torch.ao.quantization.get_default_qconfig('x86')

In [6]:
# model_fp32_prepared = model # torch.ao.quantization.prepare(model)
# input_fp32 = torch.randn(1, 1, 7539)
# model_fp32_prepared(input_fp32)


{'audio': tensor([[[-0.1373, -0.1210, -0.3083,  ..., -0.0587,  0.1481,  0.2086]]],
        grad_fn=<SliceBackward0>),
 'z': tensor([[[-2.7376, -5.9216,  0.1534,  ...,  0.0890, -1.9257, -3.3301],
          [ 3.7167, -0.2829, -2.7268,  ...,  7.0995,  2.1479, -0.1615],
          [ 4.7342, -6.3042, -4.4666,  ...,  1.9308, -3.1288,  1.8012],
          ...,
          [-0.4077,  2.9430, -6.8092,  ..., -2.7056,  1.0259, -6.5504],
          [-5.8558, -3.9825,  0.2893,  ..., -3.9748, -5.7457, -5.4240],
          [ 3.6048,  5.4514, -2.6522,  ..., -5.2044, -0.1594, -0.8426]]],
        grad_fn=<AddBackward0>),
 'codes': tensor([[[ 252,  329,  581, 1003,  207,  699,  387,  849,  123,  913,  790,
            160,  160,  807,  550],
          [  65,   47,  776,  530,  529, 1009,  942,  168,  234,  480,  132,
            799,  340,  761,  766],
          [ 769,  749,   80,  139,   70,  973,  956,  652,  829,  313,  891,
            891,  486,  121,  156],
          [ 808,  790,  918,  727,  189,  683, 

In [7]:
model_int8 = torch.ao.quantization.convert(model_fp32_prepared)

RuntimeError: Only Tensors created explicitly by the user (graph leaves) support the deepcopy protocol at the moment

In [3]:
# Quantization 
import torch
import torch_tensorrt

# from torch.quantization import quantize_dynamic, quantize_static, prepare, convert

# Stuff to try
# 1. Linear quantization r = S(q-z)
# 2. AWQ (activation-aware weight quantization)

ModuleNotFoundError: No module named 'torch_tensorrt'

In [8]:


##########SAVE##############
# Save and load to and from disk
x.save("compressed.dac")
x = dac.DACFile.load("compressed.dac")

# Decompress it back to an AudioSignal
y = model.decompress(x)

# Write to file
y.write('output.wav')

#################################


/home/gridsan/awang2/.local/lib/python3.9/site-packages/audiotools/core/audio_signal.py:501: UserWarning: PySoundFile failed. Trying audioread instead.
  data, sample_rate = librosa.load(
/home/gridsan/awang2/.local/lib/python3.9/site-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


NoBackendError: 